In [1]:
import keras_genomics
from keras_genomics.layers.convolutional import RevCompConv1D
import keras
import keras.layers as kl
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
from seqdataloader.batchproducers import coordbased
from seqdataloader.batchproducers.coordbased import coordstovals
from seqdataloader.batchproducers.coordbased import coordbatchproducers
from seqdataloader.batchproducers.coordbased import coordbatchtransformers
from keras.models import load_model
from keras.utils import CustomObjectScope
from deeplift.dinuc_shuffle import dinuc_shuffle
import pandas as pd
import json
import os
import gzip
from math import log
from matplotlib import pyplot as plt
from scipy.stats import spearmanr, pearsonr, gaussian_kde
%matplotlib inline

Using TensorFlow backend.
/users/amr1/miniconda3/envs/basepair/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/users/amr1/miniconda3/envs/basepair/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/users/amr1/miniconda3/envs/basepair/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/users/amr1/mi

In [2]:
dfs = pd.read_excel("../data/experimental/paralogous/GSE97691_Combined_Runx1_10nM_50nM_Runx2_10nM_50nM_log.xlsx")

In [3]:
dfs.head()

,ID,Sequence,Runx1_10nM,Runx1_50nM,Runx2_10nM,Runx2_50nM
0,Bound_17395,TGCCCGAAGCGCGGACGCGGTTTACGGTGAGCTGTA,8.9259,10.2641,8.5636,9.5680
1,Bound_08312,TGACAGAGGGAGAAATGTGGTGTTCTCAGATGAGGG,7.3533,8.7496,6.9991,6.7682
2,Bound_02784,CCCAGCCTCAGAGATTGTGGTCGAATGTTTGACAGT,9.5369,10.3016,9.6956,10.2241
3,Bound_00414,ATGCAGAAGGAATGCTGTGGTAGACAGCATTCCTTT,8.0527,9.4738,7.6158,7.3931
4,Bound_17565,AAGTCTACAGCTCTTTGAGGTTCATGTGTGACTCTA,7.2224,9.0434,7.4547,7.0404


In [4]:
dfs.tail()

,ID,Sequence,Runx1_10nM,Runx1_50nM,Runx2_10nM,Runx2_50nM
25447,NegCtrl_06757,CCTAAGCACTAGAGCACATAGCCTCCTTGCCCCAGG,6.8451,7.8590,6.9684,6.6662
25448,NegCtrl_06758,CGGCCTGCACTGCCTTGCAACACACGGCCCTCATCC,6.6081,7.8580,6.1172,6.3702
25449,NegCtrl_06759,ATCACCCCTTATAGTGACTTCAGCTCCTCCTTTCTC,6.7009,7.1307,7.3175,6.3824
25450,NegCtrl_06760,GTGTAACAATTGGCTAGCGCTAATCCTTTTACAGAG,6.6027,7.8039,6.3875,5.9421
25451,NegCtrl_06761,CAGGACAGAGATACAACAGAGAGAAGAGGAGCTGGG,6.9772,6.7569,7.0728,5.6673


In [5]:
all_seqs = dfs['Sequence']
all_xvals1 = dfs['Runx1_10nM']
all_xvals2 = dfs['Runx1_50nM']
sample_indices = np.random.choice(25451, 2600)

In [6]:
seqs = []
xvals1 = []
xvals2 = []
for idx in sample_indices:
    seqs.append(all_seqs[idx])
    xvals1.append(all_xvals1[idx])
    xvals2.append(all_xvals2[idx])

In [7]:
os.environ["CUDA_VISIBLE_DEVICES"]="2"
def multinomial_nll(true_counts, logits):
    """Compute the multinomial negative log-likelihood
    Args:
      true_counts: observed count values
      logits: predicted logit values
    """
    counts_per_example = tf.reduce_sum(true_counts, axis=-1)
    dist = tfp.distributions.Multinomial(total_count=counts_per_example,
                                         logits=logits)
    return (-tf.reduce_sum(dist.log_prob(true_counts)) / 
            tf.to_float(tf.shape(true_counts)[0]))

#from https://github.com/kundajelab/basepair/blob/cda0875571066343cdf90aed031f7c51714d991a/basepair/losses.py#L87
class MultichannelMultinomialNLL(object):
    def __init__(self, n):
        self.__name__ = "MultichannelMultinomialNLL"
        self.n = n

    def __call__(self, true_counts, logits):
        for i in range(self.n):
            loss = multinomial_nll(true_counts[..., i], logits[..., i])
            if i == 0:
                total = loss
            else:
                total += loss
        return total

    def get_config(self):
        return {"n": self.n}

In [8]:
with CustomObjectScope({'MultichannelMultinomialNLL': MultichannelMultinomialNLL,'RevCompConv1D': RevCompConv1D}):
    model = load_model("../data/models/k562_runx1_1_model.h5")







Instructions for updating:
Use `tf.cast` instead.


In [9]:
fastapath = "../data/genome/hg19/male.hg19.fa"
GenomeDict={}
sequence=''
inputdatafile = open(fastapath)
for line in inputdatafile:
    if line[0]=='>':
        if sequence != '':
            GenomeDict[chrm] = ''.join(sequence)
        chrm = line.strip().split('>')[1]
        sequence=[]
        Keep=False
        continue
    else:
        sequence.append(line.strip())
GenomeDict[chrm] = ''.join(sequence)

seq_len = 1346
out_pred_len = 1000
test_chrms = ["chr1", "chr8", "chr21"]
seq_peaks = []
with gzip.open("../data/paralogous/K562_runx1_1/idr.optimal_peak.narrowPeak.gz", 'rt') as inp:
    for line in inp:
        chrm = line.strip().split('\t')[0]
        if chrm not in test_chrms:
            continue
        pStart = int(line.strip().split('\t')[1])
        summit = pStart + int(line.strip().split('\t')[-1])
        start = int(summit - (seq_len/2))
        end = int(summit + (seq_len/2))
        seq_peaks.append(GenomeDict[chrm][start:end].upper())
    
ltrdict = {
           'a':[1,0,0,0],'c':[0,1,0,0],'g':[0,0,1,0],'t':[0,0,0,1],
           'n':[0,0,0,0],'A':[1,0,0,0],'C':[0,1,0,0],'G':[0,0,1,0],
           'T':[0,0,0,1],'N':[0,0,0,0]}
def getOneHot(ISM_sequences):
  # takes in list of sequences
    one_hot_seqs = []
    for seq in ISM_sequences:
        one_hot = []
        for i in range(len(seq)):
            one_hot.append(ltrdict[seq[i:i+1]])
        one_hot_seqs.append(one_hot)
    return np.array(one_hot_seqs)

In [10]:
def fill_into_center(seq, insert):
    pos = int((len(seq)-len(insert))/2.0)
    new_seq = seq[:pos] + insert + seq[pos+len(insert):]
    return new_seq

In [ ]:
from deeplift.dinuc_shuffle import dinuc_shuffle

num_samples = 100
yvals = []
for idx, insert in enumerate(seqs):
    if idx % 200 == 0:
        print("Done with ", idx)
    pre_seqs = []
    post_seqs = []
    indices = np.random.choice(len(seq_peaks), num_samples, replace=False)
    for idx in indices:
        pre_seq = dinuc_shuffle(seq_peaks[idx])
        post_seq = fill_into_center(pre_seq, insert)
        pre_seqs.append(pre_seq)
        post_seqs.append(post_seq)
    pre = model.predict([getOneHot(pre_seqs), np.zeros((num_samples,)), np.zeros((num_samples,out_pred_len,2))])
    post = model.predict([getOneHot(post_seqs), np.zeros((num_samples,)), np.zeros((num_samples,out_pred_len,2))])
    yvals.append(np.mean(post[0]-pre[0]))

Done with  0
Done with  200
Done with  400
Done with  600
Done with  800
Done with  1000
Done with  1200
Done with  1400
Done with  1600


In [ ]:
xy = np.vstack([xvals1,yvals])
z = gaussian_kde(xy)(xy)
smallFont = {'size' : 10}
plt.rc('font', **smallFont)
fig, ax = plt.subplots()
ax.scatter(xvals1, yvals, c=z, edgecolor='', alpha=0.5)
plt.xlabel("Normalized Log gcPBM Signal")
plt.ylabel("Delta Log Counts")
plt.title("measurements vs model predictions--spearman: "+ \
          str(spearmanr(xvals1, yvals)) + ", pearson: "+ str(pearsonr(xvals1, yvals)))
plt.show()

In [ ]:
xy = np.vstack([xvals2,yvals])
z = gaussian_kde(xy)(xy)
smallFont = {'size' : 10}
plt.rc('font', **smallFont)
fig, ax = plt.subplots()
ax.scatter(xvals2, yvals, c=z, edgecolor='', alpha=0.5)
plt.xlabel("Normalized Log gcPBM Signal")
plt.ylabel("Delta Log Counts")
plt.title("measurements vs model predictions--spearman: "+ \
          str(spearmanr(xvals2, yvals)) + ", pearson: "+ str(pearsonr(xvals2, yvals)))
plt.show()